In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data/gemstone.csv')
df.head()       

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df.drop(columns=['id'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   carat    193573 non-null  float64
 1   cut      193573 non-null  object 
 2   color    193573 non-null  object 
 3   clarity  193573 non-null  object 
 4   depth    193573 non-null  float64
 5   table    193573 non-null  float64
 6   x        193573 non-null  float64
 7   y        193573 non-null  float64
 8   z        193573 non-null  float64
 9   price    193573 non-null  int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 14.8+ MB


In [4]:
#independent and dependent features
X = df.drop(columns=['price'])
y = df['price']
print("Independent features:\n",X.head())
print("Dependent features\n",y.head())

Independent features:
    carat        cut color clarity  depth  table     x     y     z
0   1.52    Premium     F     VS2   62.2   58.0  7.27  7.33  4.55
1   2.03  Very Good     J     SI2   62.0   58.0  8.06  8.12  5.05
2   0.70      Ideal     G     VS1   61.2   57.0  5.69  5.73  3.50
3   0.32      Ideal     G     VS1   61.6   56.0  4.38  4.41  2.71
4   1.70    Premium     G     VS2   62.6   59.0  7.65  7.61  4.77
Dependent features
 0    13619
1    13387
2     2772
3      666
4    14453
Name: price, dtype: int64


In [5]:
# define which columns should be ordinal encoded and which should be scaled
categorical_columns = [features for features in X.columns if X[features].dtype == 'object']
numerical_columns = [features for features in X.columns if X[features].dtype in ['int64', 'float64']]
print("Categorical columns:\n", categorical_columns)
print("Numerical columns:\n", numerical_columns)

Categorical columns:
 ['cut', 'color', 'clarity']
Numerical columns:
 ['carat', 'depth', 'table', 'x', 'y', 'z']


In [6]:
df[categorical_columns]['cut'].unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [7]:
cut_categorical = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_categorical = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categorical = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

In [8]:
# importing simple imputer to handle missing values
from sklearn.impute import SimpleImputer

In [9]:
from sklearn.preprocessing import StandardScaler # handling feature scaling
from sklearn.preprocessing import OrdinalEncoder  # for ordinal encoding
# pipeline for preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer # for combining multiple preprocessing steps

In [10]:
## Numerical pipeline
num_pipeline=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # handling missing values
    ('scaler', StandardScaler())  # feature scaling
])


##categorical pipeline
cat_pipeline=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # handling missing values
    ('ordinal_encoder',OrdinalEncoder(categories=[cut_categorical,color_categorical,clarity_categorical])),  # ordinal encoding
    ('scalar', StandardScaler())  # feature scaling
]
)

In [11]:
# combining the pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numerical_columns),  # numerical pipeline
        ('cat', cat_pipeline, categorical_columns)  # categorical pipeline
    ]
)

In [12]:
## Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
#transformation of training and testing data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


In [14]:
X_train

array([[-0.82314374, -1.12998781, -0.64189666, ...,  0.87410007,
        -0.93674681,  1.35074594],
       [ 0.94502267, -1.77782269,  0.92190185, ..., -1.13764403,
         0.91085333,  0.68445511],
       [ 1.9584839 ,  0.16568195,  0.40063568, ..., -0.13177198,
         0.91085333,  0.01816428],
       ...,
       [ 0.92345966,  0.90606467,  0.40063568, ..., -0.13177198,
         0.29498662,  0.01816428],
       [-1.03877378, -0.66724861, -0.64189666, ..., -1.13764403,
         0.29498662,  2.01703677],
       [-1.03877378, -0.01941373,  0.92190185, ..., -1.13764403,
         0.29498662, -1.31441737]], shape=(135501, 9))

In [15]:
X_test

array([[-0.62907669,  0.25822979, -0.12063049, ...,  0.87410007,
        -1.55261352, -0.64812655],
       [ 2.60537405, -2.14801405, -0.12063049, ..., -1.13764403,
         0.29498662, -1.31441737],
       [-1.1250258 , -1.22253565,  0.92190185, ..., -0.13177198,
        -0.93674681,  2.01703677],
       ...,
       [-0.82314374, -0.01941373, -0.64189666, ...,  0.87410007,
         0.29498662,  2.01703677],
       [ 0.90189666, -0.66724861,  1.44316802, ..., -0.13177198,
         1.52672004, -0.64812655],
       [ 0.47063656,  0.90606467, -0.64189666, ..., -1.13764403,
        -0.93674681,  0.01816428]], shape=(58072, 9))

In [16]:
X_train=pd.DataFrame(X_train, columns=preprocessor.get_feature_names_out())

In [17]:
X_test=pd.DataFrame(X_test, columns=preprocessor.get_feature_names_out())
X_test

,num__carat,num__depth,num__table,num__x,num__y,num__z,cat__cut,cat__color,cat__clarity
0,-0.629077,0.258230,-0.120630,-0.600482,-0.581521,-0.572248,0.874100,-1.552614,-0.648127
1,2.605374,-2.148014,-0.120630,2.126042,2.198832,1.959219,-1.137644,0.294987,-1.314417
2,-1.125026,-1.222536,0.921902,-1.374347,-1.414721,-1.469110,-0.131772,-0.936747,2.017037
3,-1.017211,-0.574701,0.921902,-1.158385,-1.161138,-1.194265,-0.131772,1.526720,2.017037
4,0.858771,0.628421,-0.641897,0.947248,0.985258,1.004495,0.874100,0.910853,-0.648127
...,...,...,...,...,...,...,...,...,...
58067,0.255007,0.535873,0.921902,0.416340,0.369414,0.425874,-1.137644,1.526720,-1.314417
58068,-0.607514,0.535873,-0.641897,-0.528495,-0.554351,-0.499920,0.874100,-1.552614,0.018164
58069,-0.823144,-0.019414,-0.641897,-0.834441,-0.862273,-0.847093,0.874100,0.294987,2.017037
58070,0.901897,-0.667249,1.443168,1.046230,0.967145,0.932167,-0.131772,1.526720,-0.648127


In [18]:
X_train.head()

,num__carat,num__depth,num__table,num__x,num__y,num__z,cat__cut,cat__color,cat__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,-0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,0.910853,0.684455
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-0.131772,0.910853,0.018164
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.874100,-0.320880,2.017037
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-1.137644,1.526720,-0.648127


In [19]:
# model training
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error


In [20]:
regression= LinearRegression()
regression.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [21]:
regression.coef_

array([ 6432.97591819,  -132.34206204,   -70.48787525, -1701.38593925,
        -494.17005097,   -76.32351645,    68.80035873,  -464.67990411,
         652.10059539])

In [22]:
regression.intercept_

np.float64(3976.87873890229)

In [23]:
# automate prediction
import numpy as np
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"Mean Squared Error: {mse}")
    print(f"R^2 Score: {r2*100}")
    print(f"Mean Absolute Error: {mae}")
    print(f"Root Mean Squared Error: {rmse}")
    return mse, r2, mae, rmse
    


In [26]:
# train multiple models

model={
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet()
}

model_list=[]
r2_list=[]

for i in model.keys():
    print(f"Training {i}...")
    model[i].fit(X_train, y_train)
    evaluate_model(model[i], X_test, y_test)
    
    # store the model and r2 score
    model_list.append(model[i])
    r2_list.append(r2_score(y_test, model[i].predict(X_test)))
    print(f"{i} trained successfully!\n")
    
    print("="*50)
    
# Display the R^2 scores for each model
for i, r2 in zip(model.keys(), r2_list):
    print(f"{i} R^2 Score: {r2*100}")
    print("="*50)
    
model_zip= zip(model_list, r2_list)
min_val = min(model_zip, key=lambda x: x[1])
print("Model with minimum R^2 score:", min_val[0])

Training Linear Regression...
Mean Squared Error: 1029473.3531156846
R^2 Score: 93.62906819996049
Mean Absolute Error: 675.0758270067479
Root Mean Squared Error: 1014.6296630375463
Linear Regression trained successfully!

Training Ridge Regression...
Mean Squared Error: 1029482.8101268928
R^2 Score: 93.62900967491633
Mean Absolute Error: 675.1077629781348
Root Mean Squared Error: 1014.6343233534399
Ridge Regression trained successfully!

Training Lasso Regression...
Mean Squared Error: 1029533.1506505491
R^2 Score: 93.62869814082755
Mean Absolute Error: 676.2421173665507
Root Mean Squared Error: 1014.659130275064
Lasso Regression trained successfully!

Training ElasticNet Regression...
Mean Squared Error: 2351174.871397875
R^2 Score: 85.44967219374031
Mean Absolute Error: 1060.9432977143006
Root Mean Squared Error: 1533.3541245902313
ElasticNet Regression trained successfully!

Linear Regression R^2 Score: 93.62906819996049
Ridge Regression R^2 Score: 93.62900967491633
Lasso Regression